In [1]:
import numpy as np
import pandas as pd

from numpy.typing import NDArray

In [2]:
K = 1.9872E-3  # kcal/(mol·K)
T = 300        # k
BETA = 1/(K*T) # mol/kcal

In [3]:
class alascan_dec_ene():
    def __init__(self, fp: str) ->  None:
        self.fp = fp
        self.key = "DELTA,Total Energy Decomposition:\n"
        self.raw_col = ["Frame","Residue","Location","Int","vdW","ele","GB","nonpol","tol"]
        self.ene_col = ["Int", "vdW", "ele", "GB", "nonpol", "tol"]

    def convert_to_matrix(self, df, resid):

        split_resid = lambda _: int(_.split(" ")[-1])

        df.columns = self.raw_col
        df["resid"] = df["Residue"].apply(split_resid)
        df = df.drop(columns=["Residue", "Location"]).copy()

        df = df[df["resid"] == resid][["Frame"]+self.ene_col]
        df = df.reset_index(drop=True)
        df["Frame"] = df["Frame"].astype("int")
        for col in self.ene_col:
            df[col] = df[col].astype("float64")

        return df.to_numpy()

    def __call__(self, resid: int):
        
        with open(self.fp, "r") as F:
            ss = F.readlines()

        # Frame #,Residue,Location,Internal,van der Waals,Electrostatic,Polar Solvation,Non-Polar Solv.,TOTAL
        # +2 矫正到纯数据行
        key_start_s = []
        key_end_s = []
        flag = False
        for i, s in enumerate(ss):
            if s == self.key:
                key_start_s.append(i+2)
                flag = True
            if flag and s == "\n":
                key_end_s.append(i)
                flag = False
        assert len(key_start_s) == 2 and len(key_end_s) == 2, print("Error[iaw]>: must the dec ene for ala scan")

        df_wi = pd.DataFrame(np.genfromtxt(ss[key_start_s[0]:key_end_s[0]+1],delimiter=',',dtype=str))
        wi = self.convert_to_matrix(df_wi, resid)

        df_mut = pd.DataFrame(np.genfromtxt(ss[key_start_s[1]:key_end_s[1]+1],delimiter=',',dtype=str))
        mut = self.convert_to_matrix(df_mut, resid)

        return wi, mut




In [45]:
from rich.progress import track
from multiprocessing import Pool
from tqdm import tqdm
fps = ["12_SER","14_ASN","15_SER","16_ARG","185_PRO","186_HIE","188_PHE","189_PHE","191_PHE","192_PRO","193_VAL","194_THR"
        ,"197_ILE","199_PRO","201_TYR","20_GLN","21_VAL","236_MET","239_ASN","23_PHE","243_TYR","244_ASN","248_THR","249_VAL"
        ,"250_TYR","253_LEU","33_PRO","35_TRP","37_ASN","38_PHE","43_GLN","45_TYR","46_PRO","47_THR","48_LEU","54_ARG"
        ,"55_ARG","56_ILE","57_HIE","58_SER","59_TYR","62_HID","64_TRP","66_PHE"]


In [65]:
len(fps)

44

In [ ]:
def cal_InterEntropy_aa(wi_aa: NDArray, BETA: float):
    E_int = np.sum(2*wi_aa[:, [2, 3]], axis = 1)        # vdw_vac + ele_vac
    E_int_mean = np.mean(E_int)
    return np.log(np.mean(np.exp(BETA*(E_int - E_int_mean))))

def one_job(fp):
    try:
        resid, resn = fp.split("_")
        resid = int(resid)
        m_wi, m_mut = alascan_dec_ene("/home/iaw/DATA/IE/test/data/{}.csv".format(fp))(resid)
        ie = K*T*(cal_InterEntropy_aa(m_mut, BETA) - cal_InterEntropy_aa(m_wi, BETA))
        return resid, ie
    except Exception as e:
          print(f"Error processing {fp}: {e}")
          return None

def cal_InterEntropy_aa_2(wi_aa: NDArray, BETA: float):
    E_int = np.sum(wi_aa[:, [2, 3]], axis = 1)        # vdw_vac + ele_vac
    E_int_mean = np.mean(E_int)
    return np.log(np.mean(np.exp(BETA*(E_int - E_int_mean))))


def one_job_2(fp):
    resid, resn = fp.split("_")
    resid = int(resid)
    m_wi, m_mut = alascan_dec_ene("/home/iaw/DATA/IE/test/data/{}.csv".format(fp))(resid)
    ie = K*T*(cal_InterEntropy_aa_2(m_mut, BETA) - cal_InterEntropy_aa_2(m_wi, BETA))
    return resid, ie

In [47]:
with Pool(processes=20) as pool:
    results = pool.imap_unordered(one_job, fps)
    out = {}
    for resid, ie in track(results, total=len(fps)):
        out[resid] = ie


Output()

In [71]:
aa5 = [14, 14, 14, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 23, 23, 23, 23, 23, 23, 33, 33, 33, 33, 33, 33, 33, 33, 33, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 38, 38, 38, 38, 38, 38, 38, 38, 43, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 58, 58, 58, 58, 58, 58, 58, 58, 58, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 62, 62, 62, 62, 62, 62, 62, 62, 62, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 186, 186, 186, 186, 186, 186, 186, 186, 186, 186, 186, 186, 186, 186, 187, 187, 187, 187, 187, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 188, 189, 189, 189, 189, 189, 189, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 191, 192, 192, 192, 192, 192, 192, 192, 192, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 193, 194, 194, 194, 196, 196, 196, 196, 196, 196, 196, 196, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 197, 198, 198, 198, 198, 198, 198, 198, 199, 201, 201, 201, 201, 239, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 244, 244, 244, 244, 244, 244, 244, 244, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 249, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253, 253]
aa5 = set(aa5)

In [72]:
ie_all_1 = 0.0
for k, v in out.items():
    if k in aa5:
        ie_all_1 += v

In [75]:
ie_all_1 = 0.0
for k, v in out.items():
    if k not in [12, 21, 43, 199, 236, 239, 248]:
        ie_all_1 += v

In [73]:
len(aa5)

40

In [76]:
ie_all_1

-26.863514528635932

In [54]:
ie_all = np.array(list(out.values()))

In [55]:
np.sum(ie_all)

-26.920743288837464

In [80]:
from glob import glob
import sys, os

In [81]:
os.path.basename(glob("/home/iaw/DATA/IE/test/data" + "/" +"*.csv")[0])

'46_PRO.csv'

In [79]:
glob("/home/iaw/DATA/IE/test/data" + "/" +"*.csv")[0]

'/home/iaw/DATA/IE/test/data/46_PRO.csv'

In [57]:
out2 = {}
for fp in track(["12_SER","14_ASN","15_SER","16_ARG","185_PRO","186_HIE","188_PHE","189_PHE","191_PHE","192_PRO","193_VAL","194_THR"
        ,"197_ILE","199_PRO","201_TYR","20_GLN","21_VAL","236_MET","239_ASN","23_PHE","243_TYR","244_ASN","248_THR","249_VAL"
        ,"250_TYR","253_LEU","33_PRO","35_TRP","37_ASN","38_PHE","43_GLN","45_TYR","46_PRO","47_THR","48_LEU","54_ARG"
        ,"55_ARG","56_ILE","57_HIE","58_SER","59_TYR","62_HID","64_TRP","66_PHE"]):
    resid, resn = fp.split("_")
    resid = int(resid)
    m_wi, m_mut = alascan_dec_ene(fp = "/home/iaw/DATA/IE/test/data/{}.csv".format(fp))(resid)
    ie = K*T*(cal_InterEntropy_aa_2(m_mut, BETA) - cal_InterEntropy_aa_2(m_wi, BETA))
    
    out2[str(resid)] = ie

Output()

In [58]:
ie_all2 = np.array(list(out2.values()))

In [59]:
np.sum(ie_all2)

-4.598527208250043